In [ ]:
import sqlite3

# إنشاء اتصال بقاعدة البيانات
conn = sqlite3.connect("image_features.db")
cursor = conn.cursor()

# إنشاء جدول لتخزين السمات
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS features (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    image_name TEXT,
    keypoints BLOB,
    descriptors BLOB
)
"""
)

conn.commit()

In [ ]:
import cv2
import os
import numpy as np
import pickle

# مسار مجلد الصور
image_folder_path = "photo_product"

# إعداد SIFT
sift = cv2.SIFT_create()


# دالة لتحويل النقاط الرئيسية (keypoints) إلى تنسيق يمكن تخزينه
def keypoints_to_array(keypoints):
    keypoints_array = []
    for kp in keypoints:
        kp_as_list = (kp.pt, kp.size, kp.angle, kp.response, kp.octave, kp.class_id)
        keypoints_array.append(kp_as_list)
    return keypoints_array


# دالة لحفظ السمات في قاعدة البيانات
def save_features_to_db(image_name, keypoints, descriptors):
    keypoints_array = keypoints_to_array(keypoints)
    # استخدام pickle لتسلسل البيانات
    keypoints_blob = pickle.dumps(keypoints_array)
    descriptors_blob = pickle.dumps(descriptors)
    cursor.execute(
        """
    INSERT INTO features (image_name, keypoints, descriptors)
    VALUES (?, ?, ?)
    """,
        (image_name, keypoints_blob, descriptors_blob),
    )
    conn.commit()


# استخراج السمات من كل صورة وحفظها في قاعدة البيانات
for filename in os.listdir(image_folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_folder_path, filename)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        keypoints, descriptors = sift.detectAndCompute(image, None)
        save_features_to_db(filename, keypoints, descriptors)